In [1]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import optax
import torch
import treescope
from diffusers import StableDiffusionPipeline

from fitness import brightness
from noise_injection_pipelines import SDSamplingPipeline
from vmf.vmf_nes import vMF_NES
import optax

treescope.basic_interactive_setup()

/home/jajal/mambaforge/envs/diff-traf/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
pipeline = StableDiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    device_map=None,
    torch_dtype=torch.float16,
    use_safetensors=True,
    safety_checker=None,
).to("cuda")
pipeline.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [3]:
sample_fn = SDSamplingPipeline(
    pipeline=pipeline,
    prompt="",
    num_inference_steps=50,
    classifier_free_guidance=True,
    guidance_scale=7.5,
    generator=torch.Generator().manual_seed(0),
    add_noise=False,
    height=512,
    width=512,
)

In [4]:
sample_fn.regenerate_latents()
sample_fn.rembed_text("An orange cat sitting on a couch")

In [5]:
popsize = 16
es = vMF_NES(
    population_size=popsize, 
    solution=jnp.zeros((4, 64 ,64)),
    mean_optimizer=optax.sgd(10.0),
    kappa_optimizer=optax.sgd(10.0),
)

INFO:2025-05-18 15:40:17,027:jax._src.xla_bridge:867: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


In [6]:
# Initialize state
key = jax.random.key(0)

key, subkey, init_key = jax.random.split(key, 3)
params = es.default_params
params = params.replace(kappa_init=50.)


mean = jax.random.normal(init_key, (4, 64, 64))
state = es.init(subkey, mean, params)

In [ ]:
best_sols = []
for i in range(10):
    old_mean, old_kappa = state.mean, state.kappa
    key, key_ask, key_norm, key_tell = jax.random.split(key, 4)
    population, state = es.ask(key_ask, state, params)

    popsize = population.shape[0]
    norms = jnp.sqrt(
        jax.random.chisquare(key_norm, es.num_dims, (popsize,))
    )
    population = population * norms[:, None, None, None]

    torch_population = torch.from_dlpack(population)

    imgs = []
    for latents in torch_population.split(1):
        img = sample_fn(noise_injection=latents)
        imgs.extend(img)

    fitnesses = [brightness(img) for img in imgs]
    idx_max = np.argmax(fitnesses)
    best_img = imgs[idx_max]

    fitnesses = jnp.array(fitnesses).squeeze()
    # Tell the ES about the fitnesses
    state, metrics = es.tell(key_tell, population, fitnesses, state, params)
    # Get the best solution
    best_sol = state.best_solution
    best_sols.append((best_img, state.best_fitness))
    display(best_img)
    print(f"Iteration {i}")
    print(
        f"Best fitness: {state.best_fitness}",
        f"Mean fitness: {jnp.mean(fitnesses)}",
        f"Max Fitness: {jnp.max(fitnesses)}",
        f"Min Fitness: {jnp.min(fitnesses)}",
        f"KAppa: {state.kappa}",
        f"Delta Mean: {jnp.linalg.norm(old_mean - state.mean)}",
        f"Delta Kappa: {jnp.linalg.norm(old_kappa - state.kappa)}",
    )

GENERATING SAMPLES
GOT SAMPLES


In [ ]:
state